In [ ]:
import sc2ts.inference as si
import json
import pandas as pd
import numpy as np
import tszip

In [ ]:
rematch_data = snakemake.input[1]

ts_in = tszip.load(snakemake.input[0])
ts_out =  tszip.load(snakemake.input[2])

records = {}
with open(rematch_data) as f:
    for d in json.load(f):
        r = si.RematchRecombinantsLbsResult.fromdict(d)
        records[r.recombinant] = r

In [ ]:
tree_in = ts_in.first()
tree_out = ts_out.first()
data = []
for r in records.values():
   
    lbs = r.long_branch_split
    hmm_match = None
    if lbs is None:
        assert len(r.recomb_match.path) == 1
        hmm_match = r.recomb_match
    elif len(lbs.hmm_match.parents) == 1:
        hmm_match = lbs.hmm_match

    if hmm_match is not None:
        old_descendants = tree_in.num_samples(r.recombinant)
        new_descendants = tree_out.num_samples(r.recombinant)
        assert old_descendants == new_descendants
        data.append({
            "recombinant": r.recombinant,
            "num_descendant_samples": old_descendants,
            "rewire_existing": lbs is None,
            "original_muts": len(r.original_match.mutations),
            "new_muts": len(hmm_match.mutations),
        })
df = pd.DataFrame(data).set_index("recombinant")
df["muts_gained"] = df["new_muts"] - df["original_muts"]
df

In [ ]:
ts_in.num_trees, ts_out.num_trees

In [ ]:
ts_out.num_mutations - ts_in.num_mutations

In [ ]:
ts_out.num_nodes - ts_in.num_nodes

# Rewire existing

In [ ]:
df["rewire_existing"].value_counts()

In [ ]:
df_re = df[df["rewire_existing"]]
df_re

In [ ]:
df_re["muts_gained"].value_counts()

In [ ]:
df_re["muts_gained"].sum()

In [ ]:
df_re["num_descendant_samples"].value_counts()

In [ ]:
df_re["num_descendant_samples"].median()

In [ ]:
df_re["num_descendant_samples"].sum()

# Rewire long branch splits

In [ ]:
df_lbs = df[~df["rewire_existing"]]
df_lbs

In [ ]:
df_lbs["muts_gained"].value_counts()

## Look at majority first

In [ ]:
df_lbs = df_lbs[df_lbs["muts_gained"] >= 0]
df_lbs

In [ ]:
df_lbs["muts_gained"].value_counts()

In [ ]:
df_lbs["muts_gained"].sum()

In [ ]:
df_lbs["num_descendant_samples"].sum()

## Now the outliers

In [ ]:
df_lbs = df[~df["rewire_existing"]]
df_lbs = df_lbs[df_lbs["muts_gained"] < 0]
df_lbs